In [3]:
import numpy as np
import cv2 as cv
import glob

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# vAR WIDTH
H =  9
W =  6

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((W*H,3), np.float32)     #### alterar parametro  
objp[:,:2] = np.mgrid[0:H,0:W].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\calibragem_img"+'/*.jpg')
print(images)

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (H,W), None)   #### alterar parametro 

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (H,W), corners2, ret)        #### alterar parametro 
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print(ret, mtx, dist, rvecs, tvecs)

img = cv.imread(r'C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\calibragem_img\WIN_20250429_21_56_11_Pro.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)



# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

# salvando parametros
np.savez('calibration_data.npz', 
         ret=ret, 
         mtx=mtx, 
         dist=dist, 
         rvecs=rvecs, 
         tvecs=tvecs, 
         error=mean_error/len(objpoints))
 
print( "total error: {}".format(mean_error/len(objpoints)) )





['C:\\Users\\marce\\OneDrive\\Desktop\\Masters_degree\\2_Semestre\\VARM\\project_2\\calibragem_img\\WIN_20250429_21_56_11_Pro.jpg', 'C:\\Users\\marce\\OneDrive\\Desktop\\Masters_degree\\2_Semestre\\VARM\\project_2\\calibragem_img\\WIN_20250429_21_56_17_Pro.jpg', 'C:\\Users\\marce\\OneDrive\\Desktop\\Masters_degree\\2_Semestre\\VARM\\project_2\\calibragem_img\\WIN_20250429_21_56_20_Pro.jpg', 'C:\\Users\\marce\\OneDrive\\Desktop\\Masters_degree\\2_Semestre\\VARM\\project_2\\calibragem_img\\WIN_20250429_21_56_23_Pro.jpg', 'C:\\Users\\marce\\OneDrive\\Desktop\\Masters_degree\\2_Semestre\\VARM\\project_2\\calibragem_img\\WIN_20250429_21_56_25_Pro.jpg', 'C:\\Users\\marce\\OneDrive\\Desktop\\Masters_degree\\2_Semestre\\VARM\\project_2\\calibragem_img\\WIN_20250429_21_56_29_Pro.jpg', 'C:\\Users\\marce\\OneDrive\\Desktop\\Masters_degree\\2_Semestre\\VARM\\project_2\\calibragem_img\\WIN_20250429_21_56_32_Pro.jpg', 'C:\\Users\\marce\\OneDrive\\Desktop\\Masters_degree\\2_Semestre\\VARM\\project_2\

In [ ]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print( "total error: {}".format(mean_error/len(objpoints)) )